In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson

In [2]:
def getResponse(url, param = {}):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers, params=param)
    return response

In [3]:
# https://www.pule.com/voice/23260
# https://file2.pule.com/v2a/23260.mp3

In [10]:
# 获取音频页面的音频下载地址和对应文本
def getAudioUrlAndText(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    page_text = response.text
    #print(page_text)
    tree = etree.HTML(page_text)
    temp = tree.xpath('//div[@class="audio-playbox"]//source/@src')    
    if len(temp) == 0:
        return "", ""    
    audioSrcUrl = temp[0]
    text = tree.xpath('//div[@class="content"]/text()')
    return audioSrcUrl, text

In [11]:
url, text = getAudioUrlAndText('https://www.pule.com/voice/23260')
print(url)
print(text)

https://file2.pule.com/v2a/23260.mp3
['结膜炎是在眼睛的白眼球表面，以及眼皮内表面薄层结构，发生的炎症。\n\n结膜炎为细菌、病毒、衣原体、真菌、寄生虫等感染，风沙、烟尘、紫外线暴露以及酸碱或有毒气体化学损伤，抑或过敏性、全身性疾病包括肺结核、梅毒、甲状腺疾病等、眼睛周围组织包括角膜、巩膜、眼睑、鼻腔、鼻窦所导致结膜的炎症反应。\n\n结膜炎常见的症状有眼红、异物感、烧灼感、痒、怕光、流泪、分泌物增多，有时也会有眼痛。']


In [12]:
# 下载音频和文本
def downloadAudioAndText(url):
    audioUrl, text = getAudioUrlAndText(url)
    if audioUrl == "":
        print('error')
        return
    text = '\n'.join(text)
    fileName = (audioUrl.split('/')[-1]).split('.')[0]
    with open('text/' + fileName + '.txt', 'w', encoding='utf-8') as fp:
        fp.write(text)
    content = getResponse(audioUrl).content
    with open('mp3/' + fileName + '.mp3', 'wb') as fp:
        fp.write(content)
    print(url, "successful")

In [13]:
downloadAudioAndText('https://www.pule.com/voice/23260')

https://www.pule.com/voice/23260 successful


In [14]:
# 获取每一页的所有音频页链接
def getAudioUrlOfEveryPage(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    page_text = response.text
    tree = etree.HTML(page_text)
    allPage = tree.xpath('//div[@class="content"]/a')
    linkList = []
    for li in allPage:
        pageLink = 'https://www.pule.com' + li.xpath('./@href')[0]
        #print(pageLink)
        linkList.append(pageLink)
    return linkList

In [15]:
for link in getAudioUrlOfEveryPage('https://www.pule.com/voice/'):
    print(link)

https://www.pule.com/voice/23260
https://www.pule.com/voice/23764
https://www.pule.com/voice/24285
https://www.pule.com/voice/27259
https://www.pule.com/voice/27345
https://www.pule.com/voice/27394
https://www.pule.com/voice/27432
https://www.pule.com/voice/27841
https://www.pule.com/voice/28037
https://www.pule.com/voice/28067


In [16]:
for link in getAudioUrlOfEveryPage('https://www.pule.com/voice/'):
    downloadAudioAndText(link)

https://www.pule.com/voice/23260 successful
https://www.pule.com/voice/23764 successful
https://www.pule.com/voice/24285 successful
https://www.pule.com/voice/27259 successful
https://www.pule.com/voice/27345 successful
https://www.pule.com/voice/27394 successful
https://www.pule.com/voice/27432 successful
https://www.pule.com/voice/27841 successful
https://www.pule.com/voice/28037 successful
https://www.pule.com/voice/28067 successful


In [19]:
# 获取视频页面的视频下载地址和对应文本
def getVideoUrlAndText(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    page_text = response.text
    tree = etree.HTML(page_text)
    temp = tree.xpath('//source/@src')    
    if len(temp) == 0:
        return "", ""    
    audioSrcUrl = temp[0]
    text = tree.xpath('//div[@class="content"]/p/text()')
    return audioSrcUrl, text

In [20]:
url, text = getVideoUrlAndText('https://www.pule.com/kepu/39382')
print(url)
print(text)

https://vod.pule.com/6c992c3bvodcq1500003583/b1bcdba55285890813370524325/f0.mp4
['肾炎它分为急性肾炎和慢性肾炎，急性肾炎和慢性肾它的症状是不一样的，一般急性肾炎的症状它起病比较急，它有水肿、有血尿，而且有部分的有感染中毒的症状，有感冒的症状，还有比如有皮肤长脓疱疮、有发热，然后两个礼拜以后出现蛋白尿，出现血尿，一般是急性肾炎常见的症状。', '而慢性肾炎它的症状就比较复杂了，一般慢性肾炎它都有尿里面的改变，比如尿里面出现蛋白尿、出现血尿，如果这个出现蛋白尿、血尿三个月以上，我们平时所说的叫做CKD，这就是慢性肾炎，有些慢性肾炎它完全可以没有症状，它没有任何症状像正常人一样，他一发病到医院去检查肾功能就损害了，肌酐就增高，有尿毒症症状，严重的就需要透析治疗，这就是慢性肾炎和急性肾炎常见的症状。']


In [21]:
# 下载视频和文本
def downloadVideoAndText(url):
    audioUrl, text = getVideoUrlAndText(url)
    if audioUrl == "":
        print('error')
        return
    text = '\n'.join(text)
    fileName = (audioUrl.split('/')[-1]).split('.')[0]
    with open('video_text/' + fileName + '.txt', 'w', encoding='utf-8') as fp:
        fp.write(text)
    content = getResponse(audioUrl).content
    with open('video/' + fileName + '.mp4', 'wb') as fp:
        fp.write(content)
    print(url, "successful")

In [23]:
# 获取每一页的所有视频页链接
def getVideoUrlOfEveryPage(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    page_text = response.text
    tree = etree.HTML(page_text)
    allPage = tree.xpath('//div[@class="videoitem"]')
    linkList = []
    for li in allPage:
        pageLink = 'https://www.pule.com' + li.xpath('./a[1]/@href')[0]
        #print(pageLink)
        linkList.append(pageLink)
    return linkList

In [24]:
for link in getVideoUrlOfEveryPage('https://www.pule.com/kepu/'):
    print(link)

https://www.pule.com/kepu/39382
https://www.pule.com/kepu/35072
https://www.pule.com/kepu/35228
https://www.pule.com/kepu/37086
https://www.pule.com/kepu/36228
https://www.pule.com/kepu/35404
https://www.pule.com/kepu/36468
https://www.pule.com/kepu/37518
https://www.pule.com/kepu/38374


In [26]:
for link in getVideoUrlOfEveryPage('https://www.pule.com/kepu/'):
    downloadVideoAndText(link)

https://www.pule.com/kepu/39382 successful
https://www.pule.com/kepu/35072 successful
https://www.pule.com/kepu/35228 successful
https://www.pule.com/kepu/37086 successful
https://www.pule.com/kepu/36228 successful
https://www.pule.com/kepu/35404 successful
https://www.pule.com/kepu/36468 successful
https://www.pule.com/kepu/37518 successful
https://www.pule.com/kepu/38374 successful
